In [1]:
import pandas as pd


import spacy
import en_core_web_trf

nlp = spacy.load("en_core_web_trf")

from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

import warnings
warnings.filterwarnings('ignore') # (action='once')

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df = pd.read_json('/Users/landsiedelj/Downloads/hundredthousand_wp_merged_text_small.jsonl', lines=True, orient='records')

In [3]:
df.head(2)

,id,article_url,title,author,published_date,type,source,publish_date,orig-id,merged_total_text,text_cf
0,ABM35HDPIII6RP4GUI2RWXWOTE,/opinions/the-hollow-compassion-of-trump-wary-...,Trump’s America goes full Charles Dickens,Dana Milbank,NaN,None,None,NaN,None,Trump’s America goes full Charles Dickens. Tru...,[Trump’s America goes full Charles Dickens. Tr...
1,7f4cc4bc-351a-11e4-9e92-0899b306bbea,https://www.washingtonpost.com/local/u-md-bask...,U-Md. basketball arena is transformed into mak...,Karen Chen,1.409960e+12,article,The Washington Post,NaN,None,U-Md. basketball arena is transformed into mak...,[U-Md. basketball arena is transformed into ma...


In [5]:
# df['text_cf'] = [','.join(map(str, l)) for l in df['text_cf']]


In [4]:
# write entity in parsed_articles 
import tqdm
parsed_articles = []
for article, id in tqdm.tqdm(zip(df['merged_total_text'], df['id']), total=len(df)):
    parsed_articles.extend([[id, str(entity), entity.label_] for entity  in nlp(article).ents])

100%|██████████| 1000/1000 [09:51<00:00,  1.69it/s]


In [5]:
# create dfs from entities
df_2 = pd.DataFrame(parsed_articles, columns=['id', 'merged_total_text', 'entity_type'])

# rename columns for one df
df_2.rename(columns={'merged_total_text': 'entity'}, inplace=True)

In [6]:
# merge back to initial df
df = (pd.merge(df, df_2, on='id'))

In [7]:
df = (df.loc[df['entity_type'].isin(['PERSON'])])

In [8]:
df['entity'] = df['entity'].str.strip()
df['entity'] = df['entity'].str.replace('[\[\]\"\'\d\,\<\/]','')

In [9]:
# Replace acronyms
df['entity'] = df['entity'].str.replace('JFK','John Fitzgerald Kennedy').replace('FDR','Franklin Delano Roosevelt')

In [13]:
# TO do: write into dict
# Unification of entity
df['entity'] = df['entity'].str.replace('JFK','John Fitzgerald Kennedy').replace('FDR','Franklin Delano Roosevelt').replace('Sting','Gordon Matthew Sumner')
df['entity'] = df['entity'].str.replace("Pelosi", "Nancy Pelosi").replace("Reagan", "Ronald Reagan")
df['entity'] = df['entity'].str.replace("Christie", "Chris Christie").replace("Strasburg", "Stephen Strasburg")
df['entity'] = df['entity'].str.replace("Ovechkin", "Alexander Ovechkin").replace("Wall", "John Wall").replace("Cousins", "Kirk Cousins").replace("Boehner", "John Boehner")
df['entity'] = df['entity'].str.replace("Graham", "Lindsey Graham").replace("Comey", "James Comey").replace("Hogan", "Larry Hogan")
df['entity'] = df['entity'].str.replace("Gruden", "Jay Gruden").replace("Harris", "Kamala Harris")replace("Brady", "Tom Brady").replace("Netanyahu", "Benjamin Netanyahu")
df['entity'] = df['entity'].str.replace("McConnell", "Mitchell McConnell").replace("Mitch McConnell", "Mitchell McConnell").replace("Vladimir Wladimir Putin", "Wladimir Wladimirowitsch Putin")
df['entity'] = df['entity'].str.replace("Cheney", "Liz Cheney").replace("Bowser", "Muriel Bowser").replace("Turgeon", "Mark Turgeon")
df['entity'] = df['entity'].str.replace("Manafort", "Paul Manafort").replace("Sessions", "Jeff Sessions")
df['entity'] = df['entity'].str.replace("Giuliani", "Rudolph Giuliani").replace("Rudy Rudolph Giuliani", "Rudolph Giuliani")
df['entity'] = df['entity'].str.replace("Nixon", "Richard Nixon").replace("McCain", "John McCain").replace("McAuliffe", "Terence McAuliffe")
df['entity'] = df['entity'].str.replace("Yellen", "Janet Yellen").replace("Trotz", "Barry Trotz").replace("Scherzer", "Max Scherzer")
df['entity'] = df['entity'].str.replace("Barack Obama’s", "Barack Obama").replace("Obama’s", "Barack Obama").replace("Obama", "Barack Obama").replace("Barack Barack Obama", "Barack Obama")
df['entity'] = df['entity'].str.replace("Clinton", "Hillary Clinton").replace("Hillary Rodham Clinton", "Hillary Clinton")
df['entity'] = df['entity'].str.replace("Putin", "Wladimir Wladimirowitsch Putin").replace("Vladimir Putin", "Wladimir Wladimirowitsch Putin")
df['entity'] = df['entity'].str.replace("Cruz", "Ted Cruz").replace("Pence", "Mike Pence").replace("Woods", "Tiger Woods").replace("Zimmerman", "Ryan Zimmerman").replace("Kasich", "John Kasich")
df['entity'] = df['entity'].str.replace("Biden", "Joe Biden").replace("Mueller", "Robert Mueller").replace("Trump", "Donald Trump")
df['entity'] = df['entity'].str.replace("Kavanaugh", "Brett Kavanaugh").replace("Rubio", "Marco Rubio").replace("Buttigieg", "Pete Buttigieg").replace("Pompeo", "Mike Pompeo")
df['entity'] = df['entity'].str.replace("Maduro", "Nicolás Maduro").replace("Rizzo", "Anthony Rizzo").replace("Sondland", "Gordon Sondland")
df['entity'] = df['entity'].str.replace("Erdogan", "Recep Tayyip Erdoğan").replace("Holtby", "Braden Holtby").replace("Bannon", "Steve Bannon")
df['entity'] = df['entity'].str.replace("Cuccinelli", "Ken Cuccinelli").replace("Kenneth Ken Cuccinelli", "Ken Cuccinelli")
df['entity'] = df['entity'].str.replace("Donald Trump’s", "Donald Trump").replace("Snowden", "Edward Snowden").replace("Kushner", "Jared Kushner")
df['entity'] = df['entity'].str.replace("Alex Alexander Ovechkin", "Alexander Ovechkin").replace("Hillary Clinton’s", "Hillary Clinton")
df['entity'] = df['entity'].str.replace("Zelensky", "Wolodymyr Selenskyj").replace("Volodymyr Zelenskyy", "Wolodymyr Selenskyj")
df['entity'] = df['entity'].str.replace("Bashar al-Baschar Assad", "Bashar Assad").replace("Assad", "Bashar Assad").replace("Shanahan", "Mike Shanahan")
df['entity'] = df['entity'].str.replace("Khashoggi", "Jamal Khashoggi").replace("Castro", "Fidel Castro").replace("Tillerson", "Rex Tillerson")
df['entity'] = df['entity'].str.replace("RGIII", "Robert Griffin III")
df['entity'] = df['entity'].str.replace("Xi", "Xi Jinping").replace("Xi Xi Jinping", "Xi Jinping").replace("Xi Jinping Jinping", "Xi Jinping")
df['entity'] = df['entity'].str.replace("Macron", "Emmanuel Macron").replace("Emmanuel Emmanuel Macron", "Emmanuel Macron")
df['entity'] = df['entity'].str.replace("Gingrich", "Newton Gingrich").replace("Newton Newton Gingrich", "Newton Gingrich").replace("Newt Newton Gingrich", "Newton Gingrich")
df['entity'] = df['entity'].str.replace("Donald Donald Trump", "Donald Trump")
df['entity'] = df['entity'].str.replace("Vladimir Wladimir Putin", "Wladimir Wladimirowitsch Putin")
df['entity'] = df['entity'].str.replace("Bashar al-Baschar Assad", "Bashar Assad").replace("Assad", "Bashar Assad").replace("Kim Jong Un", "Kim Jong-un")
df['entity'] = df['entity'].str.replace("Hitler", "Adolf Hitler").str.replace("Adolf Adolf Hitler", "Adolf Hitler")
df['entity'] = df['entity'].str.replace("Volodymyr Wolodymyr Selenskyj", "Wolodymyr Selenskyj")
df['entity'] = df['entity'].str.replace("Bashar al-Assad", "Bashar Assad").replace('Roosevelt','Theodore Roosevelt')
df['entity'] = df['entity'].str.replace("Hillary Clintons", "Hillary Clinton").replace("Ivanka Donald Trump", "Ivanka Trump")
df['entity'] = df['entity'].str.replace("Spicer", "Sean Spicer").replace("Shakespeare", "William Shakespeare")
df['entity'] = df['entity'].str.replace("Santorum", "Rick Santorum").replace("Rick Santorum", "Richard Santorum")
df['entity'] = df['entity'].str.replace("Beal", "Bradley Beal").replace('Rosenstein','Rod Rosenstein')
df['entity'] = df['entity'].str.replace('Kardashian','Kim Kardashian').replace('da Gama’s','Vasco da Gama').replace('@CindyBoren on','Cindy Boren').replace('Bourdain','Anthony Bourdain')
df['entity'] = df['entity'].str.replace('Costello','Elvis Costello').replace('Romney','Mitt Romney').replace('Trudeau','Justin Trudeau').replace("Duterte","Rodrigo Duterte")
df['entity'] = df['entity'].str.replace('Franklin Delano Theodore Roosevelt','Franklin Delano Roosevelt').replace('Franklin D. Theodore Roosevelt','Franklin Delano Roosevelt')

In [10]:
df = df[df['author'] != df['entity']] # if author is in col entity -> remove, we only want "Person" from within the article

In [ ]:
df.to_pickle('/Users/landsiedelj/washington_post_bias/WP_article_NER_large_final.pkl')